In [76]:
# Install required packages.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt


def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [77]:
import torch
from torch_geometric.datasets import Amazon
from torch_geometric.utils import *

dataset = Amazon(root='/tmp/Amazon', name='Computers')
print(dataset[0])
print(dataset[0].x)
print(dataset[0].edge_index)
adj =  to_scipy_sparse_matrix(dataset[0].edge_index, edge_attr=dataset[0].edge_attr)
print(adj)

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 1., 1., 0.]])
tensor([[    0,     0,     0,  ..., 13751, 13751, 13751],
        [  507,  6551,  8210,  ..., 12751, 13019, 13121]])
  (0, 507)	1.0
  (0, 6551)	1.0
  (0, 8210)	1.0
  (0, 9745)	1.0
  (1, 184)	1.0
  (1, 206)	1.0
  (1, 337)	1.0
  (1, 638)	1.0
  (1, 872)	1.0
  (1, 1272)	1.0
  (1, 1456)	1.0
  (1, 1517)	1.0
  (1, 1559)	1.0
  (1, 2073)	1.0
  (1, 2159)	1.0
  (1, 2316)	1.0
  (1, 2396)	1.0
  (1, 2414)	1.0
  (1, 2764)	1.0
  (1, 2772)	1.0
  (1, 2791)	1.0
  (1, 2965)	1.0
  (1, 3080)	1.0
  (1, 3354)	1.0
  (1, 3957)	1.0
  :	:
  (13751, 1280)	1.0
  (13751, 1796)	1.0
  (13751, 2155)	1.0
  (13751, 2485)	1.0
  (13751, 2673)	1.0
  (13751, 4239)	1.0
  (13751, 5278)	1.0
  (13751, 6298)	1.0
  (13751, 653

In [78]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import roc_auc_score, average_precision_score
import scipy.sparse as sp
import numpy as np
import os
import time

In [79]:
class Args:
  dataset = 'amazon_computers'
  model = 'GAE'

  input_dim = 767 
  hidden1_dim = 32
  hidden2_dim = 16
  use_feature = True

  num_epoch = 200
  learning_rate = 0.01
args=Args()

In [80]:
'''
****************NOTE*****************
CREDITS : Thomas Kipf
since datasets are the same as those in kipf's implementation, 
Their preprocessing source was used as-is.
*************************************
'''
import numpy as np
import sys
import pickle as pkl
import networkx as nx
import scipy.sparse as sp

from google.colab import drive
drive.mount('/content/drive')
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/GraphNN/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import numpy as np
import scipy.sparse as sp

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)

def mask_test_edges(adj):
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    # TODO: Clean up.

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    #num_test = int(np.floor(edges.shape[0] / 60.))
    #num_val = int(np.floor(edges.shape[0] / 10.))
    num_val=int(edges.shape[0]*0.1)
    num_test=int(edges.shape[0]*0.6)
    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        # if idx_i == idx_j:
        #     continue
        # if ismember([idx_i, idx_j], edges_all):
        #     continue
        # if test_edges_false:
        #     if ismember([idx_j, idx_i], np.array(test_edges_false)):
        #         continue
        #     if ismember([idx_i, idx_j], np.array(test_edges_false)):
        #         continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        # if idx_i == idx_j:
        #     continue
        # if ismember([idx_i, idx_j], train_edges):
        #     continue
        # if ismember([idx_j, idx_i], train_edges):
        #     continue
        # if ismember([idx_i, idx_j], val_edges):
        #     continue
        # if ismember([idx_j, idx_i], val_edges):
        #     continue
        # if val_edges_false:
        #     if ismember([idx_j, idx_i], np.array(val_edges_false)):
        #         continue
        #     if ismember([idx_i, idx_j], np.array(val_edges_false)):
        #         continue
        val_edges_false.append([idx_i, idx_j])

    # assert ~ismember(test_edges_false, edges_all)
    # assert ~ismember(val_edges_false, edges_all)
    # assert ~ismember(val_edges, train_edges)
    # assert ~ismember(test_edges, train_edges)
    # assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np



class GraphConvSparse(nn.Module):
    def __init__(self, input_dim, output_dim, adj, activation = F.relu, **kwargs):
        super(GraphConvSparse, self).__init__(**kwargs)
        self.weight = glorot_init(input_dim, output_dim) 
        self.adj = adj
        self.activation = activation

    def forward(self, inputs):
        x = inputs
        x = torch.mm(x,self.weight)
        x = torch.mm(self.adj, x)
        outputs = self.activation(x)
        return outputs


def dot_product_decode(Z):
    A_pred = torch.sigmoid(torch.matmul(Z,Z.t()))
    return A_pred

def glorot_init(input_dim, output_dim):
    init_range = np.sqrt(6.0/(input_dim + output_dim))
    initial = torch.rand(input_dim, output_dim)*2*init_range - init_range
    return nn.Parameter(initial)


class GAE(nn.Module):
    def __init__(self,adj):
        super(GAE,self).__init__()
        self.base_gcn = GraphConvSparse(args.input_dim, args.hidden1_dim, adj)
        self.gcn_mean = GraphConvSparse(args.hidden1_dim, args.hidden2_dim, adj, activation=lambda x:x)

    def encode(self, X):
        hidden = self.base_gcn(X)
        z = self.mean = self.gcn_mean(hidden)
        return z

    def forward(self, X):
        Z = self.encode(X)
        A_pred = dot_product_decode(Z)
        return A_pred

In [83]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(abs(rowsum), -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    if isinstance(features, np.ndarray):
        features = sp.csr_matrix(features)
    return sparse_to_tuple(features)

In [84]:
os.environ['CUDA_VISIBLE_DEVICES'] = ""
from scipy import sparse
#adj, features = load_data(args.dataset)
adj = sparse.csr_matrix(adj)
features = dataset[0].x
# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train
# Some preprocessing
adj_norm = preprocess_graph(adj)

num_nodes = adj.shape[0]

features = preprocess_features(features.numpy().astype(float))
num_features = features[2][1]
features_nonzero = features[1].shape[0]

# Create Model
pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)


adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)


adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T), torch.FloatTensor(adj_norm[1]), torch.Size(adj_norm[2]))
adj_label = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].T),  torch.FloatTensor(adj_label[1]), torch.Size(adj_label[2]))
features = torch.sparse.FloatTensor(torch.LongTensor(features[0].T), torch.FloatTensor(features[1]), torch.Size(features[2]))


weight_mask = adj_label.to_dense().view(-1) == 1
weight_tensor = torch.ones(weight_mask.size(0)) 
weight_tensor[weight_mask] = pos_weight

# init model and optimizer
#model = getattr(model,"GAE")(adj_norm)
#model=VGAE(adj_norm)

model=GAE(adj_norm)
optimizer = Adam(model.parameters(), lr=args.learning_rate)


def get_scores(edges_pos, edges_neg, adj_rec):

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    pos = []
    for e in edges_pos:
        # print(e)
        # print(adj_rec[e[0], e[1]])
        preds.append(sigmoid(adj_rec[e[0], e[1]].item()))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:

        preds_neg.append(sigmoid(adj_rec[e[0], e[1]].data))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

def get_acc(adj_rec, adj_label):
    labels_all = adj_label.to_dense().view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy

In [ ]:
# train model
for epoch in range(1500):
    t = time.time()

    A_pred = model(features)
    optimizer.zero_grad()
    loss = log_lik = norm*F.binary_cross_entropy(A_pred.view(-1), adj_label.to_dense().view(-1), weight = weight_tensor)
    if args.model == 'VGAE':
        kl_divergence = 0.5/ A_pred.size(0) * (1 + 2*model.logstd - model.mean**2 - torch.exp(model.logstd)).sum(1).mean()
        loss -= kl_divergence

    loss.backward()
    optimizer.step()

    train_acc = get_acc(A_pred,adj_label)

    val_roc, val_ap = get_scores(val_edges, val_edges_false, A_pred)
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(loss.item()),
          "train_acc=", "{:.5f}".format(train_acc), "val_roc=", "{:.5f}".format(val_roc),
          "val_ap=", "{:.5f}".format(val_ap),
          "time=", "{:.5f}".format(time.time() - t))



test_roc, test_ap = get_scores(test_edges, test_edges_false, A_pred)
print("End of training!", "test_roc=", "{:.5f}".format(test_roc),
      "test_ap=", "{:.5f}".format(test_ap))

Epoch: 0001 train_loss= 0.72542 train_acc= 0.03343 val_roc= 0.81393 val_ap= 0.80411 time= 6.66648
Epoch: 0002 train_loss= 0.72510 train_acc= 0.00085 val_roc= 0.79860 val_ap= 0.81011 time= 5.74884
Epoch: 0003 train_loss= 0.72418 train_acc= 0.00085 val_roc= 0.79723 val_ap= 0.80951 time= 4.98660
Epoch: 0004 train_loss= 0.72234 train_acc= 0.00085 val_roc= 0.79703 val_ap= 0.80963 time= 4.67159
Epoch: 0005 train_loss= 0.71924 train_acc= 0.00085 val_roc= 0.79667 val_ap= 0.80935 time= 4.64679
Epoch: 0006 train_loss= 0.71449 train_acc= 0.00085 val_roc= 0.79577 val_ap= 0.80850 time= 4.56576
Epoch: 0007 train_loss= 0.70797 train_acc= 0.00085 val_roc= 0.79548 val_ap= 0.80825 time= 4.52849
Epoch: 0008 train_loss= 0.69983 train_acc= 0.00085 val_roc= 0.79562 val_ap= 0.80840 time= 4.56503
Epoch: 0009 train_loss= 0.69073 train_acc= 0.00085 val_roc= 0.79522 val_ap= 0.80801 time= 4.59995
Epoch: 0010 train_loss= 0.68262 train_acc= 0.00085 val_roc= 0.79506 val_ap= 0.80787 time= 4.59025
